In [6]:
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime, timedelta
import pandas as pd

# no keys required for crypto data
client = CryptoHistoricalDataClient()

years_back = 3 #Fetch 3 years of data
start_date = datetime.today() - timedelta(days=365 * years_back)
end_date = datetime.today()


request_params = CryptoBarsRequest(
                        symbol_or_symbols=["BTC/USD"],
                        timeframe=TimeFrame.Day, # Use TimeFrame.Minute for high-frequency trading
                        start=start_date,
                        end=end_date
                 )

bars = client.get_crypto_bars(request_params)

# Covert to DataFrame
df = bars.df

pd.set_option('display.width', 200)  # Set a large width
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line wrapping

# convert to dataframe
print(df)

# access bars as list - important to note that you must access by symbol key
# even for a single symbol request - models are agnostic to number of symbols
# bars["BTC/USD"]

                                        open        high           low       close       volume  trade_count          vwap
symbol  timestamp                                                                                                         
BTC/USD 2022-02-28 06:00:00+00:00  37852.380  44259.4100  37475.030000  43229.1500  1412.459176      54487.0  41250.444978
        2022-03-01 06:00:00+00:00  43224.420  45000.0000  42815.840000  44283.3600  1122.467779      47416.0  43976.369880
        2022-03-02 06:00:00+00:00  44302.030  45321.4200  43236.530000  43351.8700   844.934097      41891.0  44117.907772
        2022-03-03 06:00:00+00:00  43343.070  44103.6200  41071.000000  41381.6400   987.526989      45631.0  42387.191245
        2022-03-04 06:00:00+00:00  41379.200  41907.2100  38568.180000  39020.9600  1069.260188      42510.0  40173.915678
...                                      ...         ...           ...         ...          ...          ...           ...
        2025-02-

## Format Timestamp

In [16]:

df = df.reset_index()  # This will make timestamp a normal column
df['timestamp'] = pd.to_datetime(df['timestamp'])  # Convert column to datetime
df.set_index('timestamp', inplace=True)  # Set it back as index

# Check for missing dates
missing_dates = pd.date_range(start=start_date, end=end_date).difference(df.index)
print("Missing Dates:", missing_dates)

# Fill missing data if required
df = df.asfreq('D').fillna(method='ffill')

print(df)


Missing Dates: DatetimeIndex(['2022-02-27 17:40:44.538455', '2022-02-28 17:40:44.538455', '2022-03-01 17:40:44.538455', '2022-03-02 17:40:44.538455', '2022-03-03 17:40:44.538455', '2022-03-04 17:40:44.538455',
               '2022-03-05 17:40:44.538455', '2022-03-06 17:40:44.538455', '2022-03-07 17:40:44.538455', '2022-03-08 17:40:44.538455',
               ...
               '2025-02-17 17:40:44.538455', '2025-02-18 17:40:44.538455', '2025-02-19 17:40:44.538455', '2025-02-20 17:40:44.538455', '2025-02-21 17:40:44.538455', '2025-02-22 17:40:44.538455',
               '2025-02-23 17:40:44.538455', '2025-02-24 17:40:44.538455', '2025-02-25 17:40:44.538455', '2025-02-26 17:40:44.538455'],
              dtype='datetime64[ns]', length=1096, freq='D')
                            symbol       open        high           low       close       volume  trade_count          vwap
timestamp                                                                                                               

/var/folders/9f/dspzp56j3pjcftph22hs9gw40000gn/T/ipykernel_49792/145135840.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.asfreq('D').fillna(method='ffill')
